In [2]:
from utils.EP_api import Robot, findrobotIP
import time
robot = Robot(findrobotIP()) # router

data sent is b'robot ip 192.168.1.215' from ('192.168.1.215', 60797)
Command socket: ok


In [2]:
# robot.rotate('90')
# robot._sendcommand('chassis position ?')
robot._sendcommand('robotic_arm moveto x 182 y 0')

reply for robotic_arm moveto x 182 y 0;: ok


'ok'

In [74]:
# robot.movearm('x -100 y 100')
# robot._sendcommand('robotic_arm recenter')
robot.closearm()


reply for robotic_gripper close 1;: ok


'ok'

In [78]:
val = int(robot._sendcommand('robotic_gripper status ?'))
val

reply for robotic_gripper status ?;: 1 


1

In [2]:
robot.openarm()

reply for robotic_gripper open 1;: ok


'ok'

In [3]:
robot.exit()

Exiting...
	reply for chassis move x 0;: ok
	reply for gimbal speed p 0 y 0;: ok
	reply for robotic_arm stop;: ok
	reply for gimbal moveto p 0 y 0;: ok
	reply for robotic_arm recenter;: ok
	reply for stream off;: ok
	reply for quit;: ok
All done


In [2]:
grip_thresh = 7
def rescue_grip():
    robot._sendcommand('robotic_arm moveto x 182 y 0')
def calc_image():
    img = cv2.cvtColor(robot.frame,cv2.COLOR_BGR2GRAY)[549:625, 603:683]
    res = 0
    for row in img:
        res += sum(row)
    return res / sum([len(row) for row in img])

def set_grip_threshold():
    global grip_thresh
    rescue_grip()
    time.sleep(2)
    robot.closearm()
    time.sleep(3)
    grip_thresh = calc_image()
    time.sleep(3)
    robot.openarm()
    robot.center() 
    time.sleep(1)
    print("GRIP THRESHOLD: ",grip_thresh)
    return True


def is_gripped():
    # val = int(robot._sendcommand('robotic_gripper status ?'))
    # if val == 1: return True
    # else: return False
    val = calc_image()
    print('CURRENT THRESHOLD: ',val)
    print('COMPARISON THRESHOLD: ',grip_thresh + 3)
    if val > (grip_thresh + 3) or val < (grip_thresh - 3): return True
    else: return False

In [3]:
import cv2
robot.startvideo()

while robot.frame is None: # this is for video warm up. when frame is received, this loop is exited.
	pass

initial_setup = False
while True:
	cv2.namedWindow('Live video', cv2.WINDOW_NORMAL)
	cv2.imshow('Live video', robot.frame) # access the video feed by robot.frame
	if initial_setup is False:
            initial_setup = set_grip_threshold()
	k = cv2.waitKey(16) & 0xFF
	if k == ord('p'): # Take pic and save to screenshots
		cv2.imwrite('screenshots/last.png',robot.frame)
	if k == ord('o'): # Open claw
		robot.openarm()
	if k == ord('c'):
		robot.closearm()
	if k == ord('t'): # Test if occupied
		print('Gripped: ',is_gripped())
	if k == 27: # press esc to stop
		print("Quitting")
		robot.exit()
		break
	

reply for stream on;: ok
videothread started!
	reply for robotic_arm moveto x 182 y 0;: ok
	reply for robotic_gripper close 1;: ok
	reply for robotic_gripper open 1;: ok
	reply for gimbal moveto p 0 y 0;: ok
	reply for robotic_arm recenter;: ok
GRIP THRESHOLD:  26.28782894736842
CURRENT THRESHOLD:  87.97878289473684
COMPARISON THRESHOLD:  29.28782894736842
Gripped:  True
	reply for robotic_gripper close 1;: ok
CURRENT THRESHOLD:  26.384703947368422
COMPARISON THRESHOLD:  29.28782894736842
Gripped:  False
	reply for robotic_gripper open 1;: ok
	reply for robotic_gripper close 1;: ok
CURRENT THRESHOLD:  96.64276315789473
COMPARISON THRESHOLD:  29.28782894736842
Gripped:  True
	reply for robotic_gripper open 1;: ok
	reply for robotic_gripper close 1;: ok
CURRENT THRESHOLD:  28.694736842105264
COMPARISON THRESHOLD:  29.28782894736842
Gripped:  False
CURRENT THRESHOLD:  28.11315789473684
COMPARISON THRESHOLD:  29.28782894736842
Gripped:  False
CURRENT THRESHOLD:  28.124013157894737
COMPARIS

In [ ]:
robot.exit()